In [1]:
import numpy as np
import control as cnt
from scipy.linalg import expm
import matplotlib.pyplot as plt

In [2]:
# Esta funcion es para la ejecucion en un tiempo determinado T
# Por lo tanto todo lo que le pasemos al diccionario o arranca en 0 viene calculado de una iteracion anterior 
def pid_controller(pid_data : dict) -> dict:
    data = pid_data.copy()
    
    p = data['kp'] * (data['Beta'] * data['r'] - data['y'])
    i = data['i_futuro']
    d = (data['kd'] / (data['kd'] + data['n'] * data['h'])) * (data['d_pasado'] - data['n'] * (data['y'] - data['y_pasado']))

    data['u'] = p + i + d

    data['i_futuro'] = i + data['h'] * data['ki'] * (data['r'] - data['y'])
    data['d_pasado'] = d
    data['y_pasado'] = data['y']

    return data

In [3]:
def pid_controller_response(num: np.ndarray, den: np.ndarray, input: np.ndarray, pid_data: dict) -> np.ndarray:

    a = np.array(den).flatten()
    b = np.array(num).flatten()

    n_inicial = len(a) - 1

    y = np.zeros(len(input) + len(a) - 1)
    r = np.concatenate((np.zeros(n_inicial), input), axis = None)
    u = np.concatenate((np.zeros(n_inicial), input), axis = None)

    data = pid_data.copy()

    for k in range(n_inicial, len(u)):

        data['y'] = y[k - 1] # ADC
        data['r'] = r[k] # ADC

        data = pid_controller(data)

        u[k] = data['u'] # DAC

        for m in range(0, len(b)):
            y[k] = y[k] + b[m] * u[k - m]

        for n in range(1, len(a)):
            y[k] = y[k] - a[n] * y[k - n]
        
        y[k] = y[k] / a[0]

    return y[n_inicial - 1:]
